In [11]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
def loadpm():
    pmimg=[]
    idx=1
    while True:
        idx=str(idx)
        imgpath='pics/type/grass/40x40/poke'+idx+'.jpg'
        
        try:
            img=plt.imread(imgpath)
        except:
            break
        if(idx=='1'):
            pmimg=img.reshape(1,img.shape[0], img.shape[1], 3)
        else:
            pmimg =np.vstack((pmimg,img.reshape(1,img.shape[0], img.shape[1], 3)))
        idx=int(idx)
        idx+=1
    return pmimg

In [12]:
pokemon=loadpm()
pokemon=(pokemon-127.5)/127.5
print((pokemon.shape))

(108, 40, 40, 3)


In [3]:
import os
import numpy as np
import tensorflow as tf
from IPython.core.debugger import Tracer
from keras.layers import Input, Dense, Reshape, Flatten, Dropout,Conv2D,MaxPooling2D,Conv2DTranspose
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import Adam,RMSprop
import keras.backend as K
from keras.engine.topology import Layer
import matplotlib.pyplot as plt
from keras.layers.merge import _Merge

class Subtract(_Merge):
    def _merge_function(self, inputs):
        output = inputs[0]
        for i in range(1, len(inputs)):
            output = output-inputs[i]
        return output
class GradNorm(Layer):
    def __init__(self, **kwargs):
        super(GradNorm, self).__init__(**kwargs)

    def build(self, input_shapes):
        super(GradNorm, self).build(input_shapes)

    def call(self, inputs):
        target, wrt = inputs
        grads = K.gradients(target, wrt)
        assert len(grads) == 1
        grad = grads[0]
        return K.sqrt(K.sum(K.batch_flatten(K.square(grad)), axis=1, keepdims=True))

    def compute_output_shape(self, input_shapes):
        return (input_shapes[1][0], 1)

C:\Users\a1091\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\a1091\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\a1091\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\a1091\Anaconda3\envs\tensorflow-gpu\l

In [16]:
class Generator():
    def __init__(self,gen_dim,output_size):
        self.network=Sequential()
        element=1
        for i in output_size:
            element*=i
            
        IsDense=False
        if(IsDense):
        #Dense
            self.network.add(Dense(element//16,input_dim=gen_dim,activation='relu'))
            self.network.add(Dense(element//4,activation='relu'))
            self.network.add(Dense(element//4,activation='relu'))
            self.network.add(Dense(element//1,activation='tanh'))
            self.network.add(Reshape(output_size))
        else:
        #Convolution
            self.network.add(Dense(element//16*256,input_dim=gen_dim,activation='relu'))
            self.network.add(Reshape((output_size[0]//4,output_size[1]//4,output_size[2]*256)))
            self.network.add(Conv2DTranspose(filters=256,kernel_size=(3,3),strides=2,padding='same',activation='relu'))  
            self.network.add(Conv2DTranspose(filters=256,kernel_size=(3,3),strides=2,padding='same',activation='relu'))
            self.network.add(Conv2DTranspose(filters=3,kernel_size=(5,5),padding='same',activation='tanh'))
        self.network.summary()
class Discriminator():
    #Critic
    def __init__(self,input_size):
        self.network=Sequential()
        element=1
        for i in input_size:
            element*=i

        self.network.add(Conv2D(filters=256,kernel_size=(5,5),padding='same',input_shape=input_size))
        self.network.add(LeakyReLU(0.2))
        self.network.add(Conv2D(filters=256,kernel_size=(3,3),padding='same'))
        self.network.add(LeakyReLU(0.2))
        
        self.network.add((Flatten()))
        self.network.add(Dropout(0.2))
        #self.network.add(Dense(50))
        self.network.add(Dense(1,activation='linear'))
        
        self.network.summary()
        
        
        
class GAN():
    def __init__(self,trainingset,gen_dim):
        self.trainshape=trainingset.shape
        self.trainingset=trainingset
        self.gen_dim=gen_dim
        
        self.n_discriminator = 5
        
        self._lambda = 10
        optimizer = Adam(lr=0.0001,beta_1=0,beta_2=0.9)#RMSprop(lr=0.00005)
        #gen_dis
        self.generator=Generator(gen_dim,(trainingset.shape[1],trainingset.shape[2],trainingset.shape[3]))
        self.discriminator=Discriminator((trainingset.shape[1],trainingset.shape[2],trainingset.shape[3]))
        self.GAN_input=Input((gen_dim,))
        self.GAN_output=self.discriminator.network(self.generator.network(self.GAN_input))
        self.network=Model(self.GAN_input,self.GAN_output)
        self.network.compile(loss=self.wasserstein_loss,optimizer=optimizer)
        
        #gradient
        self.inputshape=self.trainshape[1:]
        self.real_input,self.gen_input,self.mixed_input=Input(self.inputshape),Input(self.inputshape),Input(self.inputshape)
        
        self.sub=Subtract()([self.discriminator.network(self.gen_input),self.discriminator.network(self.real_input)])
        self.grad_norm=GradNorm()([self.discriminator.network(self.mixed_input),self.mixed_input])
        self.grad=Model([self.gen_input,self.real_input,self.mixed_input],[self.sub,self.grad_norm])
        self.grad.compile(optimizer=optimizer,loss=[self.mean_loss,'MSE'],loss_weights=[1,self._lambda])
    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)
    def mean_loss(self, y_true, y_pred):
        return K.mean(y_pred)
    def train(self,iterations):
        #train discriminator
        #self.start=0
        self.batch_size=20
        #self.more_to_stop=0
        self.real_label=np.ones((self.batch_size, 1))
        self.gen_label=-np.ones((self.batch_size, 1))
        for step in range(iterations):
        #train discriminator
            for i in range(self.n_discriminator):
            #prepare img
                self.random_gen_vectors=np.random.normal(size=(self.batch_size,self.gen_dim))
                self.gen_img=self.generator.network.predict(self.random_gen_vectors)
                self.epsilon=np.random.uniform(0,1,size=(self.batch_size,1,1,1))
                self.idx=np.random.randint(0,self.trainingset.shape[0],self.batch_size)
                self.real_img=self.trainingset[self.idx]
                self.mixed=self.epsilon*self.real_img+(1-self.epsilon)*self.gen_img
                
        
                #self.labels -= 0.1 * np.random.rand(self.labels.shape[0],self.labels.shape[1])
                self.discriminator.network.trainable = True
                self.d_loss = self.grad.train_on_batch([self.gen_img,self.real_img,self.mixed],[np.ones((self.batch_size,1)),np.ones((self.batch_size,1))])
                
                
        #train generator
            self.discriminator.network.trainable = False
            self.random_gen_vector=np.random.normal(size=(self.batch_size,self.gen_dim))
            self.g_loss=self.network.train_on_batch(self.random_gen_vector,self.gen_label)
            print('iter:%d,d_loss=%f,g_loss=%f'%(step,self.d_loss[0],self.g_loss))
            if step%50==0:
                #save image
                random_vector=np.random.normal(size=(25,self.gen_dim))
                images=self.generator.network.predict(random_vector)
                images=(images+1)*127.5
                plt.figure(figsize=(10, 10))
                for i in range(images.shape[0]):
                    plt.subplot(5,5,i+1)
                    image = images[i, :, :, :]
                    image = np.reshape(image,(self.trainshape[1:]))
                    plt.imshow(image.astype(np.uint8))
                    plt.axis('off')
                plt.tight_layout()
                plt.savefig("output/poke_%d.png" % step)
                plt.close('all')
                    


In [ ]:
gan=GAN(pokemon,20)
gan.train(20000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 76800)             1612800   
_________________________________________________________________
reshape_4 (Reshape)          (None, 10, 10, 768)       0         
_________________________________________________________________
conv2d_transpose_10 (Conv2DT (None, 20, 20, 256)       1769728   
_________________________________________________________________
conv2d_transpose_11 (Conv2DT (None, 40, 40, 256)       590080    
_________________________________________________________________
conv2d_transpose_12 (Conv2DT (None, 40, 40, 3)         19203     
Total params: 3,991,811
Trainable params: 3,991,811
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


C:\Users\a1091\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


iter:0,d_loss=-7.685755,g_loss=-0.033776
iter:1,d_loss=-105.952026,g_loss=-1.660573
iter:2,d_loss=-123.182800,g_loss=-5.759989
iter:3,d_loss=-122.300056,g_loss=-11.145782
iter:4,d_loss=-113.054764,g_loss=-20.454748
iter:5,d_loss=-101.907097,g_loss=-29.164745
iter:6,d_loss=-84.962067,g_loss=-40.724129
iter:7,d_loss=-73.001831,g_loss=-51.261951
iter:8,d_loss=-60.111969,g_loss=-59.392433
iter:9,d_loss=-44.780437,g_loss=-64.621338
iter:10,d_loss=-40.434864,g_loss=-63.609154
iter:11,d_loss=-34.844349,g_loss=-58.374519
iter:12,d_loss=-30.307404,g_loss=-54.642242
iter:13,d_loss=-31.538033,g_loss=-51.570984
iter:14,d_loss=-31.725334,g_loss=-40.623650
iter:15,d_loss=-33.113579,g_loss=-31.119370
iter:16,d_loss=-36.499165,g_loss=-21.374039
iter:17,d_loss=-39.989807,g_loss=-14.675853
iter:18,d_loss=-41.575264,g_loss=-4.431133
iter:19,d_loss=-42.323700,g_loss=1.657034
iter:20,d_loss=-43.764450,g_loss=3.315390
iter:21,d_loss=-49.348030,g_loss=1.186437
iter:22,d_loss=-40.832710,g_loss=2.203045
iter:2

iter:185,d_loss=-13.159391,g_loss=-99.633003
iter:186,d_loss=-14.057295,g_loss=-87.370880
iter:187,d_loss=-13.767578,g_loss=-90.959084
iter:188,d_loss=-10.174692,g_loss=-102.519188
iter:189,d_loss=-13.309009,g_loss=-85.721642
iter:190,d_loss=-13.684922,g_loss=-104.048813
iter:191,d_loss=-14.682049,g_loss=-86.088966
iter:192,d_loss=-13.149967,g_loss=-102.783005
iter:193,d_loss=-12.444973,g_loss=-93.288986
iter:194,d_loss=-13.033555,g_loss=-116.398056
iter:195,d_loss=-11.860394,g_loss=-106.478195
iter:196,d_loss=-11.854500,g_loss=-99.894066
iter:197,d_loss=-13.066435,g_loss=-104.168076
iter:198,d_loss=-12.270594,g_loss=-109.867874
iter:199,d_loss=-13.619677,g_loss=-99.959854
iter:200,d_loss=-13.854608,g_loss=-112.190292
iter:201,d_loss=-12.673324,g_loss=-111.710915
iter:202,d_loss=-13.308568,g_loss=-105.872253
iter:203,d_loss=-15.368922,g_loss=-95.341965
iter:204,d_loss=-13.429768,g_loss=-146.923187
iter:205,d_loss=-14.178229,g_loss=-100.725464
iter:206,d_loss=-13.125933,g_loss=-102.2820

iter:364,d_loss=-13.154729,g_loss=-450.831360
iter:365,d_loss=-10.599329,g_loss=-435.791901
iter:366,d_loss=-12.229684,g_loss=-443.330017
iter:367,d_loss=-11.953853,g_loss=-428.148987
iter:368,d_loss=-10.294530,g_loss=-428.018158
iter:369,d_loss=-9.847290,g_loss=-445.923492
iter:370,d_loss=-9.234388,g_loss=-472.755066
iter:371,d_loss=-10.584282,g_loss=-461.871979
iter:372,d_loss=-11.484734,g_loss=-454.125183
iter:373,d_loss=-11.071257,g_loss=-453.649750
iter:374,d_loss=-10.783833,g_loss=-461.031433
iter:375,d_loss=-11.707830,g_loss=-446.244446
iter:376,d_loss=-13.289780,g_loss=-456.269775
iter:377,d_loss=-9.604707,g_loss=-472.576172
iter:378,d_loss=-11.401791,g_loss=-484.356781
iter:379,d_loss=-10.332222,g_loss=-466.062561
iter:380,d_loss=-12.062513,g_loss=-454.252441
iter:381,d_loss=-12.667760,g_loss=-491.857971
iter:382,d_loss=-12.355183,g_loss=-482.356293
iter:383,d_loss=-12.407598,g_loss=-470.874451
iter:384,d_loss=-10.930805,g_loss=-477.582825
iter:385,d_loss=-12.210072,g_loss=-47

In [6]:

import numpy as np
from IPython.core.debugger import Tracer
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential,Model,load_model

import keras.backend as K
from keras.engine.topology import Layer
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def wasserstein_loss(y_true, y_pred):
        return K.mean(y_true * y_pred)
gan=load_model('eevee2_model.h5', custom_objects={'wasserstein_loss': wasserstein_loss})
#gan.summary()
generator=gan.layers[1]

In [7]:
from keras.layers import Input
from ipywidgets import FloatSlider
x1=x2=x3=x4=x5=x6=x7=x8=x9=x10=x11=x12=x13=x14=x15=x16=x17=x18=x19=x20=x21=x22=x23=x24=x25=x26=x27=x28=x29=x30=0
def showimg(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25):
    input_vector=np.array([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25])
    input_vector=np.expand_dims(input_vector[0:15],axis=0)
    img=generator.predict(input_vector)
    img=np.reshape(img,(52,52,3))
    img=(img+1)*127.5
    plt.imshow(img.astype(np.uint8))
    plt.axis('off')


x1=FloatSlider(min=-5.0, max=5.0, step=0.01)
x2=FloatSlider(min=-5.0, max=5.0, step=0.01)
x3=FloatSlider(min=-5.0, max=5.0, step=0.01)
x4=FloatSlider(min=-5.0, max=5.0, step=0.01)
x5=FloatSlider(min=-5.0, max=5.0, step=0.01)
x6=FloatSlider(min=-5.0, max=5.0, step=0.01)
x7=FloatSlider(min=-5.0, max=5.0, step=0.01)
x8=FloatSlider(min=-5.0, max=5.0, step=0.01)
x9=FloatSlider(min=-5.0, max=5.0, step=0.01)
x10=FloatSlider(min=-5.0, max=5.0, step=0.01)
x11=FloatSlider(min=-5.0, max=5.0, step=0.01)
x12=FloatSlider(min=-5.0, max=5.0, step=0.01)
x13=FloatSlider(min=-5.0, max=5.0, step=0.01)
x14=FloatSlider(min=-5.0, max=5.0, step=0.01)
x15=FloatSlider(min=-5.0, max=5.0, step=0.01)
x16=FloatSlider(min=-5.0, max=5.0, step=0.01)
x17=FloatSlider(min=-5.0, max=5.0, step=0.01)
x18=FloatSlider(min=-5.0, max=5.0, step=0.01)
x19=FloatSlider(min=-5.0, max=5.0, step=0.01)
x20=FloatSlider(min=-5.0, max=5.0, step=0.01)
x21=FloatSlider(min=-5.0, max=5.0, step=0.01)
x22=FloatSlider(min=-5.0, max=5.0, step=0.01)
x23=FloatSlider(min=-5.0, max=5.0, step=0.01)
x24=FloatSlider(min=-5.0, max=5.0, step=0.01)
x25=FloatSlider(min=-5.0, max=5.0, step=0.01)
x26=FloatSlider(min=-5.0, max=5.0, step=0.01)
x27=FloatSlider(min=-5.0, max=5.0, step=0.01)
x28=FloatSlider(min=-5.0, max=5.0, step=0.01)
x29=FloatSlider(min=-5.0, max=5.0, step=0.01)
x30=FloatSlider(min=-5.0, max=5.0, step=0.01)
w=[x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25]
from ipywidgets import Layout, Button, Box,VBox,HBox,interactive_output
boxes=[]
for i in range(7):
    j=4*i+4
    if j>25:
        boxes.append(w[24])
    else:
        boxes.append(Box(w[j-4:j]))
ui=boxes[0]
for i in range(1,7):
    ui=VBox([ui,boxes[i]])

        #Box(children=w.children[j-4:j],layout=Layout(display='flex',flex_flow='row',align_items='stretch',border='solid',width='100%'))

out=interactive_output(showimg,{'x1':x1
,'x2':x2
,'x3':x3
,'x4':x4
,'x5':x5
,'x6':x6
,'x7':x7
,'x8':x8
,'x9':x9
,'x10':x10
,'x11':x11
,'x12':x12
,'x13':x13
,'x14':x14
,'x15':x15
,'x16':x16
,'x17':x17
,'x18':x18
,'x19':x19
,'x20':x20
,'x21':x21
,'x22':x22
,'x23':x23
,'x24':x24
,'x25':x25})
display(ui,out)




Output()

In [75]:
for i in range(1,31):
    print("HBox"%(i))

x1=FloatSlider(min=-5.0, max=5.0, step=0.01)
x2=FloatSlider(min=-5.0, max=5.0, step=0.01)
x3=FloatSlider(min=-5.0, max=5.0, step=0.01)
x4=FloatSlider(min=-5.0, max=5.0, step=0.01)
x5=FloatSlider(min=-5.0, max=5.0, step=0.01)
x6=FloatSlider(min=-5.0, max=5.0, step=0.01)
x7=FloatSlider(min=-5.0, max=5.0, step=0.01)
x8=FloatSlider(min=-5.0, max=5.0, step=0.01)
x9=FloatSlider(min=-5.0, max=5.0, step=0.01)
x10=FloatSlider(min=-5.0, max=5.0, step=0.01)
x11=FloatSlider(min=-5.0, max=5.0, step=0.01)
x12=FloatSlider(min=-5.0, max=5.0, step=0.01)
x13=FloatSlider(min=-5.0, max=5.0, step=0.01)
x14=FloatSlider(min=-5.0, max=5.0, step=0.01)
x15=FloatSlider(min=-5.0, max=5.0, step=0.01)
x16=FloatSlider(min=-5.0, max=5.0, step=0.01)
x17=FloatSlider(min=-5.0, max=5.0, step=0.01)
x18=FloatSlider(min=-5.0, max=5.0, step=0.01)
x19=FloatSlider(min=-5.0, max=5.0, step=0.01)
x20=FloatSlider(min=-5.0, max=5.0, step=0.01)
x21=FloatSlider(min=-5.0, max=5.0, step=0.01)
x22=FloatSlider(min=-5.0, max=5.0, step=0.0